# Estimation prix occasion véhicule

Dans le rapport, expliciter la partie métier du use case : qu'est-ce qui a de l'importance ? Problèmes rencontrés ? Choix métiers ? Pourquoi ces choix techniques ? Pourquoi ces algos ? les essais réalisés ?

Prix véhicule neuf (amener nouvelles données, autoplus.fr), options, chevaux vapeur, finition, etc)

## Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

import re

pd.set_option('display.max_colwidth', 200)
TRIGGER_VISUALIZATION = False

## Dataset(s)

In [2]:
cars_data = 'Data_cars.csv'

df = pd.read_csv(cars_data)
display(df.shape,
        df.head())

(166695, 9)

,Price,Make,Model,Model_year,Mileage,Fuel,Gearbox,Online,Description
0,11220.0,PEUGEOT,308,2014,94341.0 km,Diesel,mécanique,06/07/2018 à 3h47,"modele: 308 (2E GENERATION), version: 1.6 HDI FAP 92CH BUSINESS 5P, puissance_fiscale: 4, portes: 5.0, options: banquette arrière 3 places;volant cuir;cache bagages;airbag frontal;banquette 1/3 - ..."
1,57526.0,BMW,X6,2015,39051.0 km,Diesel,automatique,03/04/2018 à 16h41,"modele: X6 F16, version: (F16) XDRIVE30D 258 M SPORT BVA8, puissance_fiscale: 16, portes: 5.0, options: triangle de presignalisation et trousse de premiers secours;volant cuir;pack alu;affichage t..."
2,80379.0,AUDI,RS6,2014,75381.0 km,Essence,automatique,30/07/2018 à 1h55,"modele: RS6 (3E GENERATION) AVANT, version: III AVANT 4.0 TFSI 560 QUATTRO TIPTRONIC, puissance_fiscale: 47, portes: 5.0, options: banquette arrière 3 places;écran tactile;témoin de bouclage des c..."
3,2830.0,FORD,KA,2007,92282.0 km,Essence,mécanique,09/07/2018 à 14h12,"modele: KA, version: 1.3 70 FUN, puissance_fiscale: 5, portes: 3.0, options: direction assistée;jantes alu;rétroviseurs électriques et dégivrants;vitres électriques;fermeture électrique;climatisat..."
4,12925.0,FIAT,TIPO,2018,10.0 km,Essence,mécanique,19/05/2018 à 3h52,"modele: TIPO 2 SW, version: II SW 1.4 95 POP, puissance_fiscale: 5, portes: 5.0, options: Volant alu & cuir;Bluetooth inclut musique en streaming, connexion téléphone;Rétroviseurs extérieurs chauf..."


## Extraction des descriptions

In [3]:
df = pd.concat([df, df['Description'].str.extract(
    'modele: (?P<Modele>.*?), version: (?P<Version>.*?), puissance_fiscale: (?P<Puissance_fiscale>.*?), portes: (?P<Portes>.*?), options: (?P<Options>.*?), couleur: (?P<Couleur>.*?$)'
)], axis=1, sort=False)
df.head(1)

,Price,Make,Model,Model_year,Mileage,Fuel,Gearbox,Online,Description,Modele,Version,Puissance_fiscale,Portes,Options,Couleur
0,11220.0,PEUGEOT,308,2014,94341.0 km,Diesel,mécanique,06/07/2018 à 3h47,"modele: 308 (2E GENERATION), version: 1.6 HDI FAP 92CH BUSINESS 5P, puissance_fiscale: 4, portes: 5.0, options: banquette arrière 3 places;volant cuir;cache bagages;airbag frontal;banquette 1/3 - ...",308 (2E GENERATION),1.6 HDI FAP 92CH BUSINESS 5P,4,5.0,banquette arrière 3 places;volant cuir;cache bagages;airbag frontal;banquette 1/3 - 2/3;sièges rang 2 rabattables à plat;vitres teintées;boucliers av et ar couleur caisse;phares av. de jour à LED;...,BLANC BANQUISE


## Conversion en donnée numérique et Nettoyage

In [4]:
df.Model_year = df.Model_year.astype('int64')
df.Mileage = df.Mileage.str.replace(' km', '').astype('float64')
df.Puissance_fiscale = df.Puissance_fiscale.astype('int64')
df.Portes = df.Portes.replace('', np.NaN).astype('float64')
df.dtypes

Price                float64
Make                  object
Model                 object
Model_year             int64
Mileage              float64
Fuel                  object
Gearbox               object
Online                object
Description           object
Modele                object
Version               object
Puissance_fiscale      int64
Portes               float64
Options               object
Couleur               object
dtype: object

### Exploration 

In [5]:
df[['Make', 'Price']].groupby(['Make']).mean()

,Price
Make,
ABARTH,20481.254237
AC,87696.000000
AIXAM,8980.285714
ALFA ROMEO,20088.910834
ALPINA,53933.800000
...,...
TESLA,77700.300000
TOYOTA,15090.286418
TRIUMPH,28952.470588


### Prix suivant la marque
(Prix plafonnées à 200 000€)

In [7]:
if TRIGGER_VISUALIZATION:
    fig = plt.figure(figsize=(100,10))

    ax1 = sns.violinplot('Make', "Price", data=df[df.Price < 200000])
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)


### Prix suivant l'année de sortie
(Prix plafonnées à 200 000€)

In [8]:
if TRIGGER_VISUALIZATION:
    fig = plt.figure(figsize=(100,10))

    ax1 = sns.violinplot('Model_year', "Price", data=df[df.Price < 200000])
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)

### Kilométrage suivant la marque
(Prix plafonnées à 200 000€)

In [9]:
if TRIGGER_VISUALIZATION:
    fig = plt.figure(figsize=(100,10))

    ax1 = sns.violinplot('Make', "Mileage", data=df[df.Price < 200000])
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)

### Extraction des informations de version

In [15]:
df.Version.str.extract('(?P<first>^.*?$) (?P<puissance>\d{2})CH (?P<last>.*?$)')

,first,puissance,last
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
166690,NaN,NaN,NaN
166691,NaN,NaN,NaN
166692,NaN,NaN,NaN
166693,NaN,NaN,NaN


### Extraction de l'Age du véhicule au moment de la vente
année de mise en ligne de l'annonce - année de sortie

In [16]:
Online_year = df.Online.str.extract('(?P<Jour>\d{2})/(?P<Mois>\d{2})/(?P<Annee>\d{4})')['Annee'].astype('int64')
df['Model_age'] = Online_year - df['Model_year']

### (OPTIONNEL) Extraire cylindrée et Chevaux Vapeur

### Get Dummies (Fuel, Gearbox ?)

In [17]:
# Fuel
df = pd.concat([df, pd.get_dummies(df['Fuel'])], axis=1, sort=False)

In [18]:
# Gearbox
df = pd.concat([df, pd.get_dummies(df['Gearbox'])], axis=1, sort=False)

## Feature Selection

In [19]:
df[['Make', 'Model', 'Mileage', 'Description', 'Modele', 
             'Version', 'Puissance_fiscale',
             'Portes', 'Options', 'Couleur', 'Model_age',
             'Bicarburation essence GNV', 'Bicarburation essence GPL',
             'Bicarburation essence bioéthanol', 'Diesel', 'Electrique', 'Essence',
             'Hybride diesel électrique', 'Hybride essence électrique',
             'automatique', 'mécanique']].head(1)

,Make,Model,Mileage,Description,Modele,Version,Puissance_fiscale,Portes,Options,Couleur,...,Bicarburation essence GNV,Bicarburation essence GPL,Bicarburation essence bioéthanol,Diesel,Electrique,Essence,Hybride diesel électrique,Hybride essence électrique,automatique,mécanique
0,PEUGEOT,308,94341.0,"modele: 308 (2E GENERATION), version: 1.6 HDI FAP 92CH BUSINESS 5P, puissance_fiscale: 4, portes: 5.0, options: banquette arrière 3 places;volant cuir;cache bagages;airbag frontal;banquette 1/3 - ...",308 (2E GENERATION),1.6 HDI FAP 92CH BUSINESS 5P,4,5.0,banquette arrière 3 places;volant cuir;cache bagages;airbag frontal;banquette 1/3 - 2/3;sièges rang 2 rabattables à plat;vitres teintées;boucliers av et ar couleur caisse;phares av. de jour à LED;...,BLANC BANQUISE,...,0,0,0,1,0,0,0,0,0,1


In [20]:
X_FEATURE = ['Make', 'Model', 'Mileage', 'Description', 'Modele', 
             'Version', 'Puissance_fiscale',
             'Portes', 'Model_age',
             'Bicarburation essence GNV', 'Bicarburation essence GPL',
             'Bicarburation essence bioéthanol', 'Diesel', 'Electrique', 'Essence',
             'Hybride diesel électrique', 'Hybride essence électrique',
             'automatique', 'mécanique'] # 'Options', 'Couleur'
Y_TARGET = ['Price']

#DummyfiedX_DF = df[X_FEATURE]

#for col in DummyfiedX_DF.dtypes[DummyfiedX_DF.dtypes == 'object'].index:
#    for_dummy = DummyfiedX_DF.pop(col)
#    DummyfiedX_DF = pd.concat([DummyfiedX_DF, pd.get_dummies(for_dummy, prefix=col)], axis=1)

#DummyfiedX_DF.head()

X_train, X_test, y_train, y_test = train_test_split(df[X_FEATURE], df[Y_TARGET], test_size=0.33, random_state=42)

## Model Selection
Faire un model simple et mesurer sa perf avec MAPE (Mean absolute performance error)
One hot encoding ou
dummyfication car il n'y a que 6 variables différentes dans la colonne fuel

Pour matcher prix neuf d'un autre dataset, on concatene make et model et on compare avec make model du dataset autoplus.fr. Comparaison avec `fuzz.token_set_ratio`.

Voir dataframe mapper : sklearn_pandas DataFrameMapper, sklearn.pipeline Pipeline, sklearn.decomposition PCA

Parser options: tf idf, traitement fréquentiel.


## MAPE